## Repo launches at mybinder.org

https://archive.analytics.mybinder.org/

In [1]:
import pandas as pd
from sqlite_utils import Database

In [2]:
db = Database("mybinder_archive.db")

In [3]:
# get table names
db.table_names()

['mybinderlaunch', 'repo']

In [4]:
launch_table = 'mybinderlaunch'

In [5]:
# db[launch_table].columns
db[launch_table].columns_dict

{'timestamp': str,
 'version': int,
 'origin': str,
 'provider': str,
 'spec': str,
 'org': str,
 'ref': str,
 'resolved_ref': str,
 'r2d_version': str,
 'repo_url': str,
 'repo_id': str}

In [6]:
# NOTE: this notebook wont have same output (nbs_same_output)
query = f'SELECT * FROM {launch_table} ORDER BY RANDOM() LIMIT 100'
pd.read_sql_query(query, db.conn)

,timestamp,version,origin,provider,spec,org,ref,resolved_ref,r2d_version,repo_url,repo_id
0,2019-07-02T14:53:00,3,gke.mybinder.org,GitHub,bokeh/bokeh-notebooks/master,bokeh,master,,jupyter/repo2docker:4b0e8381,https://github.com/bokeh/bokeh-notebooks,5
1,2020-01-18T08:54:00,3,ovh.mybinder.org,GitHub,rani420/cool/master,rani420,master,,jupyter/repo2docker:0.10.0-127.gd9335cf,https://github.com/rani420/cool,34912
2,2019-03-13T07:17:00,1,mybinder.org,GitHub,junpark25/emath-jupyternotebooks/master,junpark25,master,,jupyter/repo2docker:9766c954,https://github.com/junpark25/emath-jupyternote...,10797
3,2020-03-24T18:13:00,3,gesis.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:0.11.0-24.gfce6488,https://github.com/ipython/ipython-in-depth,1
4,2019-08-23T18:56:00,3,gke.mybinder.org,GitHub,pyviz-demos/glaciers/jsignell/jupyter-panel-se...,pyviz-demos,jsignell/jupyter-panel-server,,jupyter/repo2docker:9c559d2a,https://github.com/pyviz-demos/glaciers,16729
...,...,...,...,...,...,...,...,...,...,...,...
95,2019-08-23T09:10:00,3,gke.mybinder.org,GitHub,ines/spacy-io-binder/live,ines,live,,jupyter/repo2docker:9c559d2a,https://github.com/ines/spacy-io-binder,25
96,2020-01-02T06:21:00,3,notebooks.gesis.org,GitHub,quantopian/qgrid-notebooks/master,quantopian,master,,jupyter/repo2docker:0.10.0-123.gf79bf04,https://github.com/quantopian/qgrid-notebooks,49
97,2019-10-21T17:12:00,3,gke.mybinder.org,GitHub,binder-examples/requirements/master,binder-examples,master,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/binder-examples/requirements,165
98,2020-06-10T21:00:00,3,turing.mybinder.org,GitHub,binder-examples/r/master,binder-examples,master,,jupyter/repo2docker:0.11.0-87.g8d85a51,https://github.com/binder-examples/r,17


### Total number of launches

In [7]:
db[launch_table].count

10140585

First timestamp

In [8]:
launches = db[launch_table].rows_where(order_by="timestamp")
first_launch_timestamp = next(launches)["timestamp"]
del launches

first_launch_timestamp

'2018-11-03T00:00:00'

Last timestamp

In [9]:
launches = db[launch_table].rows_where(order_by="timestamp desc")
last_launch_timestamp = next(launches)["timestamp"]
del launches

last_launch_timestamp

'2020-07-29T23:59:00'

### Launches per origin

In [10]:
query = f'SELECT origin, count(origin) FROM {launch_table} GROUP BY "origin";'
launches_per_origin = pd.read_sql_query(query, db.conn)
launches_per_origin

,origin,count(origin)
0,binder.mybinder.ovh,598
1,binder.mybinder.turing.ac.uk,137
2,gesis.mybinder.org,852211
3,gke.mybinder.org,5046450
4,gke.mybinder.org:443,1
5,mybinder.org,2741924
6,notebooks.gesis.org,671093
7,ovh.mybinder.org,568783
8,turing.mybinder.org,259388


In [11]:
launches_per_origin_dict = {"gke": 0, "gesis": 0, "turing": 0, "ovh": 0}
for index, row in launches_per_origin.iterrows():
    if row["origin"] == "mybinder.org":
        launches_per_origin_dict["gke"] += row["count(origin)"]
    for origin in launches_per_origin_dict.keys():
        if origin in row["origin"]:
            # print(origin, row["origin"])
            launches_per_origin_dict[origin] += row["count(origin)"]

assert sum(launches_per_origin_dict.values()) == db[launch_table].count

pd.DataFrame.from_dict(
    launches_per_origin_dict, 
    orient="index", 
    columns=["launches"]
).sort_values(by=["launches"], ascending=False)

,launches
gke,7788375
gesis,1523304
ovh,569381
turing,259525


### Launches per provider

In [12]:
query = f'SELECT provider, count(provider) FROM {launch_table} GROUP BY "provider";'
launches_per_provider = pd.read_sql_query(query, db.conn).sort_values(by=["count(provider)"], ascending=False)

launches_per_provider

,provider,count(provider)
4,GitHub,9880400
2,Gist,145219
3,Git,71185
5,GitLab,41907
7,Zenodo,1038
6,Hydroshare,395
1,Figshare,309
0,Dataverse,132


In [13]:
# percentage of first 2 providers
(sum(launches_per_provider.head(2)["count(provider)"])*100)/sum(launches_per_provider["count(provider)"])

98.86627842476544

### Repos per provider

In [14]:
query = f'SELECT provider, count(DISTINCT repo_url) FROM {launch_table} GROUP BY "provider";'
repos_per_provider = pd.read_sql_query(query, db.conn).\
                        sort_values(by=["count(DISTINCT repo_url)"], ascending=False)

repos_per_provider

,provider,count(DISTINCT repo_url)
4,GitHub,51354
3,Git,10640
2,Gist,1595
5,GitLab,939
7,Zenodo,92
6,Hydroshare,39
1,Figshare,30
0,Dataverse,15


In [15]:
# # this returns 197 repos for Git provider because they have github url but Git as provider, 
# # which is okay and should work 
# query = f'SELECT provider, count(DISTINCT repo_id) FROM {launch_table} GROUP BY "provider";'
# repos_per_provider = pd.read_sql_query(query, db.conn).\
#                         sort_values(by=["count(DISTINCT repo_id)"], ascending=False)

# repos_per_provider

### Organisations/Users in repos

In [16]:
# number of different orgs
query = "select count(distinct org) from mybinderlaunch;"
db.conn.execute(query).fetchone()[0]

35153